# Use Keras + TensorFlow to train a Neural Network
### to do Nonlinear Regression: remove bilinear (ASC) noise from DARM


In [ ]:
%matplotlib notebook
from __future__ import division
import os
import numpy as np
from scipy.io import loadmat, savemat
import scipy.signal as sig
from timeit import default_timer as timer

from keras.models import Sequential
from keras.layers import Dense, Dropout

from NonlinearRegression.tools.bilinearHelper import (cost_filter,
                                                      downsample,
                                                      get_cbc,
                                                      load_data,
                                                      normalize,
                                                      plot_cost_asd,
                                                      plot_results,
                                                      plot_training_progress,
                                                      set_plot_style,
                                                      prepare_inputs)

# Hush tensorflow warnings about AVX instructions
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
set_plot_style()

## Adjustable Parameters

In [ ]:
datafile = '../../../MockData/DARM_with_bilinear.mat'
# Training data
val_frac = 1 / 4  # Amount of data to save for validation
fs_slow  = 32     # Resample seismic data to this freq
Tchunk   = 1 / 4  # Seconds of data used to predict each DARM sample
Tbatch   = 1      # How many seconds of DARM per gradient update
Nepoch   = 3      # Number of times to iterate over training data set
# Neural Network
Nlayers    = 9       # Number of fully connected layers
cost_func  = 'mse'   # Mean Squared Error, i.e. PSD reduction
optimizer  = 'adam'  # Seems to work well...
activation = 'elu'   # "Exponential linear unit"
dropout    = 0.05    # maybe this helps in training
verbose    = 1       # Print training progress to stderr
# Cost function
zero_freqs = [6, 130]
zero_order = [10, 2]
pole_freqs = [12, 70]
pole_order = [9, 3]
# Output data and plots
tfft      = 8
save_data = False
# Whether to look for previously saved downsampled data
doLoadDownsampled = True

## Data preparation

In [ ]:
if doLoadDownsampled:
    print('Using previous downsampled data!')
    # Load up downsampled datas
    filename = 'DARM_with_bilinear_downsampled.mat'
    datas    = loadmat(filename)

    if datas['fs_slow'][0][0] != fs_slow:
        raise ValueError("Decimated sampling rate from previously saved "
                         " data (%.0f) different from requested (%.0f)" %
                         (datas['fs_slow'][0][0], fs_slow))

    angular   = datas['angular']
    beam_spot = datas['beam_spot']
    Npairs    = angular.shape[0]

    tar_raw = datas['tar_raw']
    bg_raw  = datas['bg_raw']
    tar     = datas['tar'][0]
    bg      = datas['bg'][0]

    scale       = datas['scale'][0][0]
    invBP       = datas['invBP']
    fs          = datas['fs'][0][0]
    fs_slow     = datas['fs_slow'][0][0]
    down_factor = int(fs // fs_slow)

else:
    # Load up datas
    datafile = '../../../MockData/DARM_with_bilinear.mat'
    bg_raw, tar_raw, wit, fs = load_data(datafile)

    BP, invBP = cost_filter(fs, zero_freqs, pole_freqs, zero_order,
                            pole_order)

    print("Filtering and Decimating...")
    # remove mean and normalize to std for nicer NN learning
    tar, scale = normalize(tar_raw, filter=BP)
    bg, _      = normalize(bg_raw,  filter=BP, scale=scale)

    # Get the witness signals ready for training.
    Npairs = wit.shape[0] // 2  # How many ASC + beam spot pairs

    # Shape the ASC control signal with the same filter as DARM
    beam_spot, _ = normalize(wit[:Npairs])
    angular, _   = normalize(wit[Npairs:], filter=BP)

    # Since we only care about the slow beam spot motion, we
    # don't need full rate information. Decimating the signal
    # reduces the input vector length and the number of neurons
    # we have to train.

    down_factor = int(fs // fs_slow)
    beam_spot = downsample(beam_spot, down_factor)

    # save downsampled datas
    downsampled_datafile = 'DARM_with_bilinear_downsampled.mat'
    datas = {}
    datas['angular'] = angular
    datas['beam_spot'] = beam_spot
    datas['fs'] = fs
    datas['fs_slow'] = fs_slow
    datas['tar_raw'] = tar_raw
    datas['bg_raw'] = bg_raw
    datas['tar'] = tar
    datas['bg'] = bg
    datas['scale'] = scale
    datas['invBP'] = invBP

    savemat(downsampled_datafile, datas, do_compression=True)
        
nfft = tfft * fs

# How many DARM samples are saved for validation
Nval = int(tar_raw.size * val_frac)

# How many witness samples are used to predict each DARM sample
Nchunk = int(Tchunk * fs)
Nbatch = int(Tbatch * fs)

Nang = Nchunk
Nspot = Nchunk // down_factor

In [ ]:
# Visualize cost function as an ASD
plot_cost_asd(tar, bg, fs, nfft)

In [ ]:
# Select training and validation data segments
training_target = tar[:-Nval]
training_spt    = beam_spot[:, :-Nval // down_factor]
training_ang    = angular[:, :-Nval]

validation_target = tar[-Nval:]
validation_bg     =  bg[-Nval:]
validation_spt    = beam_spot[:, -Nval // down_factor:]
validation_ang    = angular[:, -Nval:]

# Create stacked, strided input arrays
training_input = prepare_inputs(training_spt, training_ang, Nchunk)
validation_input = prepare_inputs(validation_spt, validation_ang, Nchunk)

# Minimum loss is achieved when target - prediction = bg
# Thus, MSE = mean(bg**2), i.e. var(bg)
minLoss = np.var(bg)

# Rescale validation data back to DARM units
validation_darm = validation_target * scale
validation_bg  *= scale

## Construct the neural network and train it

In [ ]:
# define the network topology  -- -- -- - - -  -  -   -   -    -
input_shape = (Npairs * (Nang + Nspot),)
model = Sequential()
model.add(Dense(input_shape[0], input_shape=input_shape,
                activation='linear'))

# this layer increases training time but not increase performance
model.add(Dense(input_shape[0], activation=activation))
model.add(Dropout(dropout))

# add layers; decrease size of each by half
layer_sizes = range(1, Nlayers)
layer_sizes.reverse()
for k in layer_sizes:
    model.add(Dense(2**k, activation=activation))
    model.add(Dropout(dropout))

model.add(Dense(1, activation='linear'))
model.compile(optimizer=optimizer, loss=cost_func)

In [ ]:
print("Starting Network learning...")
print('Best achievable cost: {:.5g}'.format(minLoss))
t_start = timer()

roomba = model.fit(training_input, training_target,
                   validation_data = (validation_input, validation_target),
                   batch_size      = Nbatch,
                   epochs          = Nepoch,
                   verbose         = verbose)

print(str(round(timer() - t_start)) + " seconds for Training.")
plot_training_progress(roomba, minLoss=minLoss)

## Generate final validation prediction

In [ ]:
# Applying model to input data...
validation_out = model.predict(validation_input, batch_size=Nbatch)[:, 0]
validation_out *= scale  # Scale to DARM units

# Unwhitening target and output data...")
validation_darm = sig.sosfilt(invBP, validation_darm)
validation_bg   = sig.sosfilt(invBP, validation_bg)
validation_out  = sig.sosfilt(invBP, validation_out)

## Plot results, print model summary

In [ ]:
plot_results(validation_darm, validation_out, validation_bg, fs, nfft)

In [ ]:
model.summary()

In [ ]:
# Saving model and processing params...
if save_data:
    model.save('FF_RegressionModel.h5')
    output_data = {
        'history': roomba.history,
        'invBP': invBP,
        'scale': scale,
        'fs': fs,
        'nfft': nfft,
    }
    savemat('Results_TFregression.mat', output_data, do_compression=True)